In [2]:
from keras import backend as k
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers.core import Flatten
from keras.datasets import mnist
from keras.utils import np_utils
from keras.optimizers import SGD, RMSprop, Adam
import numpy as np
import matplotlib.pyplot as plt
import cv2
from os import listdir
from os.path import isfile,join
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [3]:
def accuracy_metrics(Y_pred, Y_test, X_test):
    score = model.evaluate(X_test, Y_test, verbose=VERBOSE)
    count = 0
    tn = 0
    tp = 0
    fn = 0
    fp = 0
    l = 0
    for i in range(len(Y_pred)):
        pl = Y_pred[i]
        tl = Y_test[i]
        if(tl[0]==1.0):
            l = 0
        else:
            l = 1
        if(l==pl):
            count = count+1
            if(pl==1):
                tp = tp+1
            else:
                tn = tn+1
        else:
            if(l==1):
                fn = fn+1
            else:
                fp = fp+1
    print("Test score: ", score[0])
    print("Test accuracy: ", score[1])
    print("Correctly classified images: ",count)
    print("True Negatives: ", tn)
    print("True Positives: ", tp)
    print("False Positives: ", fp)
    print("False Negatives: ",fn)
    print("Precision: ",tp/(tp+fp))
    print('Recall: ',tp/(tp+fn))

In [4]:
# load the training and testing data

def load_data():
    # load train folder
    train_path = os.getcwd()+'/train'
    train_images = [f for f in listdir(train_path) if isfile(join(train_path,f))]
    X_train = []
    Y_train = []
    train_name = []
    for image in train_images:
        #load image and add it to a list -- X_train
        img = cv2.imread(train_path+'/'+image)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        X_train.append(img)
        
        #also add its label to another list -- Y_train
        if 'not' in image:
            Y_train.append(0)
        else:
            Y_train.append(1)
            
        #add its name to another list -- train_name
        train_name.append(image)
        
    tuple1 = (X_train, Y_train, train_name)
    
    test_path = os.getcwd()+'/test'
    test_images = [f for f in listdir(test_path) if isfile(join(test_path,f))]
    X_test = []
    Y_test = []
    test_name = []
    for image in test_images:
        #load image and add it to a list -- X_test
        img = cv2.imread(test_path+'/'+image)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        X_test.append(img)
        
        #also add its label to another list -- Y_test
        if 'not' in image:
            Y_test.append(0)
        else:
            Y_test.append(1)
        
        #add its name to another list -- test_name
        test_name.append(image)
        
    tuple2 = (X_test, Y_test, test_name)
    
    return tuple1, tuple2

In [5]:
#define the ConvNet
class LeNet:
    @staticmethod
    def build(input_shape, classes):
        model = Sequential()
        
        # CONV => RELU => POOL
        model.add(Conv2D(20, kernel_size=5, padding='same', input_shape=input_shape))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        
        # CONV => RELU => POOL
        model.add(Conv2D(50, kernel_size=5, border_mode="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
        
        #Flatten => RELU layers
        model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation("relu"))
        
        #a softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        return model

In [6]:
#Network and Training

NB_EPOCH = 50
BATCH_SIZE = 128
VERBOSE = 1
OPTIMIZER = Adam()
VALIDATION_SPLIT = 0.2
IMG_ROWS, IMG_COLS = 64, 64
NB_CLASSES = 2
INPUT_SHAPE = (1, IMG_ROWS, IMG_COLS)

In [7]:
#data shuffled and split between train and test set

a,b = load_data()
X_train = np.array(a[0])
Y_train = np.array(a[1])
X_test = np.array(b[0])
Y_test = np.array(b[1])
# (X_train, Y_train), (X_test, Y_test) = load_data()
k.set_image_dim_ordering("th")
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

#we need 60K * [1*28*28] shape as input to the CONVNET
X_train = X_train[:,np.newaxis,:,:]
X_test = X_test[:,np.newaxis,:,:]
print(X_train.shape[0], 'train_samples')
print(X_test.shape[0], 'test samples')

#convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(Y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(Y_test, NB_CLASSES)

1000 train_samples
469 test samples


In [8]:
#Initialize the optimizer and model
model = LeNet.build(input_shape=INPUT_SHAPE, classes=NB_CLASSES)
model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])
history = model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=NB_EPOCH, verbose=VERBOSE, 
                    validation_split=VALIDATION_SPLIT)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, padding="same", kernel_size=5)`
  del sys.path[0]


Train on 800 samples, validate on 200 samples
Epoch 1/50
800/800 [==============================] - 28s 35ms/step - loss: 0.9770 - acc: 0.4925 - val_loss: 0.7069 - val_acc: 0.4400
Epoch 2/50
800/800 [==============================] - 24s 30ms/step - loss: 0.6876 - acc: 0.5150 - val_loss: 0.6857 - val_acc: 0.4400
Epoch 3/50
800/800 [==============================] - 24s 30ms/step - loss: 0.6702 - acc: 0.5200 - val_loss: 0.6390 - val_acc: 0.7500
Epoch 4/50
800/800 [==============================] - 24s 30ms/step - loss: 0.5668 - acc: 0.8125 - val_loss: 0.4457 - val_acc: 0.9000
Epoch 5/50
800/800 [==============================] - 24s 30ms/step - loss: 0.2835 - acc: 0.9513 - val_loss: 0.2540 - val_acc: 0.9050
Epoch 6/50
800/800 [==============================] - 24s 30ms/step - loss: 0.1496 - acc: 0.9487 - val_loss: 0.1773 - val_acc: 0.9650
Epoch 7/50
800/800 [==============================] - 24s 30ms/step - loss: 0.0844 - acc: 0.9725 - val_loss: 0.1692 - val_acc: 0.9650
Epoch 8/50
800/8

In [9]:
Y_pred = model.predict_classes(X_test)
accuracy_metrics(Y_pred, Y_test, X_test)

469/469 [==============================] - 7s 14ms/step
Test score:  0.358504069854132
Test accuracy:  0.9275053307445827
Correctly classified images:  435
True Negatives:  211
True Positives:  224
False Positives:  19
False Negatives:  15
Precision:  0.9218106995884774
Recall:  0.9372384937238494
